In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import scipy as sc
import matplotlib.pyplot as plt

import itertools
from sklearn.metrics import confusion_matrix

import operator

%matplotlib inline
import numpy as np
import pandas as np
import scipy
import matplotlib.pyplot as plt
from scipy import stats
import math

from python_utils import *
import pandas as pd
import numpy as np
import math
import random
#from __future__ import division

# Functions


In [2]:
def bootstrap_ci(df_aux, variable, div, classes, repetitions = 3000, alpha = 0.05, random_state=np.arange(1,30000,10)):
    # df: a data frame that includes observations of the two sample
    # variable: the column name of the column that includes observations
    # classes: the column name of the column that includes group assignment (This column should contain two different group names)
    # repetitions: number of times you want the bootstrapping to repeat. Default is 1000.
    # alpha: likelihood that the true population parameter lies outside the confidence interval. Default is 0.05. 
    # random_stata: enable users to set their own random_state, default is None. 
    
    
    # keep the variable that we want to study and the label to be able to divide the population.
    df = df_aux[[variable, classes]]
    bootstrap_sample_size = len(df) 
    
    mean_diffs = []
    print("sample size: ", int(bootstrap_sample_size/div))
    for i in range(repetitions):
        # Sample patients randomly 
        bootstrap_sample = df.sample(n = int(bootstrap_sample_size/div), replace = True, random_state = random_state[i])
         # Mean
        mean_sample = bootstrap_sample.groupby(classes).sum() / bootstrap_sample.groupby(classes).count()
        mean_diff = mean_sample.iloc[1,0] - mean_sample.iloc[0,0]
        mean_diffs.append(mean_diff)

    # confidence interval
    left = np.percentile(mean_diffs, alpha/2*100)
    right = np.percentile(mean_diffs, 100-alpha/2*100)    
    out = [round(left,2), round(right,2)]
    
    return out

# Load data

In [3]:
def one_hot_encoder(X_static):
    X_static = pd.concat(
        [
            X_static[["Age", "Gender", "SAPSIIIScore", "MonthOfAdmission", "YearOfAdmission"]], 
            pd.get_dummies(X_static.Origin, prefix='Origin'), 
            pd.get_dummies(X_static.ReasonAdmission, prefix='ReasonAdmission'), 
            pd.get_dummies(X_static.PatientCategory, prefix='PatientCategory')
        ],
        axis=1
    )
    
    return X_static


i = 0
n = 4

X_train = pd.read_csv("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/X_train_static_" + str(n)+ ".csv", index_col=0)

y_train = pd.read_csv("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/y_train_" + str(n)+ ".csv", index_col=0)

X_val = pd.read_csv("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/X_val_static_" + str(n)+ ".csv", index_col=0)

y_val = pd.read_csv("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/y_val_" + str(n)+ ".csv", index_col=0)


X = pd.concat([X_train, X_val], axis=0)
y = y_train.append(y_val)

X = one_hot_encoder(X)

/tmp/ipykernel_651479/2125786485.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y = y_train.append(y_val)


In [4]:
X = pd.concat([X_train, X_val], axis=0)
X

,Age,Gender,SAPSIIIScore,MonthOfAdmission,YearOfAdmission,Origin,ReasonAdmission,PatientCategory
0,-0.089553,0.804829,-0.129789,-0.105610,-1.891012,2.0,2.0,2.0
1,-0.680868,0.804829,-0.129789,0.171988,-1.891012,1.0,5.0,2.0
2,-0.943675,0.804829,-0.129789,0.171988,-1.891012,3.0,4.0,1.0
3,1.027375,-1.242500,-0.129789,0.171988,-1.891012,4.0,7.0,1.0
4,-1.403586,0.804829,-0.129789,0.171988,-1.891012,2.0,8.0,1.0
...,...,...,...,...,...,...,...,...
500,1.487287,-1.242500,0.206576,-0.938404,1.696713,16.0,7.0,1.0
501,1.027375,0.804829,1.234671,-0.383208,1.696713,18.0,13.0,2.0
502,0.961674,0.804829,0.275115,-0.383208,1.696713,2.0,12.0,2.0
503,-0.877973,0.804829,-1.438377,-0.383208,1.696713,4.0,11.0,1.0


# Code

In [5]:
params = X.keys().values
X["individualMRGerm"] = y.individualMRGerm.values # Label values


percentage_resampling = 0.8
importances_variables = np.zeros((len(params), 2))
for i in range(len(params)):
    importances_variables[i,:] = bootstrap_ci(X, params[i], (1 / percentage_resampling),
                                              'individualMRGerm', repetitions=3000)

sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020


In [6]:
df_aux = pd.DataFrame(importances_variables)
df_aux["selected"] = "Y"
logicExpression = (df_aux[0] == 0) | (df_aux[1] == 0)   
df_aux["selected"] = df_aux["selected"].where(~logicExpression, "N")
logicExpression = np.sign(df_aux[0]) != np.sign(df_aux[1])
df_aux["selected"] = df_aux["selected"].where(~logicExpression, "N")
df_aux["features"] = params

In [7]:
df_aux.to_csv("results_bootstrapping_static.csv")